<a href="https://colab.research.google.com/github/lxhowl/playground/blob/main/Fix_of_pde_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()

--2023-10-24 00:39:18--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py.3’

helper.py.3         100%[===================>]   7.00K  --.-KB/s    in 0s      

2023-10-24 00:39:18 (60.2 MB/s) - ‘helper.py.3’ saved [7171/7171]

IDAES found! No need to install.
Ipopt found! No need to install.
ipopt was successfully installed
k_aug was successfuly installed
cbc was successfuly installed
clp was successfuly installed
bonmin was successfuly installed
couenne was successfuly installed
ipopt_l1 was successfuly installed
 


## Model

Here we consider a minimal PDE problem:

$$
\begin{align*}
\alpha \frac{\partial y}{\partial t} & = \beta \frac{\partial^2 y}{\partial x^2} \\
\end{align*}
$$

Initial conditions for $y(t, x)$:

$$
\begin{align*}
y(0, x) & = 0 & 0 < x \leq 1 \\
\end{align*}
$$

Boundary conditions for $y(t, x)$:

$$
\begin{align*}
y(t, 0) & = Y_s & t \geq 0 \\
\frac{dy}{dx}(t, 1) & = 0 & t \geq 0 \\
\end{align*}
$$

In [ ]:
import pyomo.environ as pyo
import pyomo.dae as dae
from idaes.core.util import DiagnosticsToolbox

In [ ]:
def pde_model(discretizer="finite_difference"):
    # parameters
    tf = 1.00
    Ys = 10.0
    alpha = 5.0
    beta = 5.0

    m = pyo.ConcreteModel()

    m.t = dae.ContinuousSet(bounds=(0, tf))
    m.x = dae.ContinuousSet(bounds=(0, 1))

    m.y = pyo.Var(m.t, m.x)
    m.s = pyo.Var(m.t, m.x)

    m.dydt = dae.DerivativeVar(m.y, wrt=m.t)
    m.dydx = dae.DerivativeVar(m.y, wrt=m.x)
    m.d2ydx2 = dae.DerivativeVar(m.y, wrt=(m.x, m.x))

    @m.Constraint(m.t, m.x)
    def pde(m, t, x):
        return beta * m.dydt[t, x]  == alpha *  m.d2ydx2[t, x]

    @m.Constraint(m.t)
    def bc1(m, t):
        return m.y[t, 0] == Ys

    @m.Constraint(m.t)
    def bc2(m, t):
        return m.dydx[t, 1] == 0

    @m.Constraint(m.x)
    def ic(m, x):
        if x == 0:
            return pyo.Constraint.Skip
        return m.y[0, x] == 0.0

    # transform and solve
    if discretizer == "finite_difference":
        pyo.TransformationFactory('dae.finite_difference').apply_to(m, wrt=m.x, nfe=8)
        pyo.TransformationFactory('dae.finite_difference').apply_to(m, wrt=m.t, nfe=10)
    else:
        pyo.TransformationFactory('dae.collocation').apply_to(m, wrt=m.x, nfe=8, ncp=1)
        pyo.TransformationFactory('dae.collocation').apply_to(m, wrt=m.t, nfe=10, ncp=1)

    return m

In [ ]:
model = pde_model()
#pyo.SolverFactory('ipopt').solve(model).write()
dt = DiagnosticsToolbox(model)
dt.report_structural_issues()

Model Statistics

        Activated Blocks: 1 (Deactivated: 0)
        Free Variables in Activated Constraints: 385 (External: 0)
            Free Variables with only lower bounds: 0
            Free Variables with only upper bounds: 0
            Free Variables with upper and lower bounds: 0
        Fixed Variables in Activated Constraints: 0 (External: 0)
        Activated Equality Constraints: 384 (Deactivated: 0)
        Activated Inequality Constraints: 0 (Deactivated: 0)
        Activated Objectives: 0 (Deactivated: 0)

------------------------------------------------------------------------------------
2 WARNINGS

        Under-Constrained Set: 4 variables, 2 constraints
        Over-Constrained Set: 3 variables, 4 constraints

------------------------------------------------------------------------------------
1 Cautions

    Caution: 110 unused variables (0 fixed)

------------------------------------------------------------------------------------
Suggested next steps:

    d

In [ ]:
dt.display_underconstrained_set()
dt.display_overconstrained_set()

Dulmage-Mendelsohn Under-Constrained Set

    Independent Block 0:

        Variables:

            d2ydx2[0,0]
            dydt[0,0]

        Constraints:

            pde[0,0]

    Independent Block 1:

        Variables:

            d2ydx2[0,0.125]
            dydt[0,0.125]

        Constraints:

            pde[0,0.125]

Dulmage-Mendelsohn Over-Constrained Set

    Independent Block 0:

        Variables:

            dydx[0,1]
            y[0,0.875]
            y[0,1]

        Constraints:

            bc2[0]
            ic[0.875]
            ic[1]
            dydx_disc_eq[0,1]



In [ ]:
model = pde_model("collocation")
#pyo.SolverFactory('ipopt').solve(model).write()
dt = DiagnosticsToolbox(model)
dt.report_structural_issues()

Model Statistics

        Activated Blocks: 1 (Deactivated: 0)
        Free Variables in Activated Constraints: 385 (External: 0)
            Free Variables with only lower bounds: 0
            Free Variables with only upper bounds: 0
            Free Variables with upper and lower bounds: 0
        Fixed Variables in Activated Constraints: 0 (External: 0)
        Activated Equality Constraints: 395 (Deactivated: 0)
        Activated Inequality Constraints: 0 (Deactivated: 0)
        Activated Objectives: 0 (Deactivated: 0)

------------------------------------------------------------------------------------
2 WARNINGS

        Under-Constrained Set: 2 variables, 1 constraints
        Over-Constrained Set: 73 variables, 84 constraints

------------------------------------------------------------------------------------
1 Cautions

    Caution: 110 unused variables (0 fixed)

------------------------------------------------------------------------------------
Suggested next steps:

   

In [ ]:
dt.display_underconstrained_set()
dt.display_overconstrained_set()

Dulmage-Mendelsohn Under-Constrained Set

    Independent Block 0:

        Variables:

            d2ydx2[0,0]
            dydt[0,0]

        Constraints:

            pde[0,0]

Dulmage-Mendelsohn Over-Constrained Set

    Independent Block 0:

        Variables:

            dydx[0,1]
            y[0,0.875]
            y[0,1]
            dydt[0.1,0.875]
            y[0.1,0.875]
            d2ydx2[0.1,0.875]
            dydx[0.1,1]
            y[0.1,1]
            dydt[0.1,1]
            d2ydx2[0.1,1]
            dydt[0.2,0.875]
            y[0.2,0.875]
            d2ydx2[0.2,0.875]
            dydx[0.2,1]
            y[0.2,1]
            dydt[0.2,1]
            d2ydx2[0.2,1]
            dydt[0.3,0.875]
            y[0.3,0.875]
            d2ydx2[0.3,0.875]
            dydx[0.3,1]
            y[0.3,1]
            dydt[0.3,1]
            d2ydx2[0.3,1]
            dydt[0.4,0.875]
            y[0.4,0.875]
            d2ydx2[0.4,0.875]
            dydx[0.4,1]
            y[0.4,1]
        

## PDE initial and boundary inclusion problem

### Using finite difference

PDE

$$
\begin{align*}
\alpha \frac{\partial y}{\partial t} & = \beta \frac{\partial^2 y}{\partial x^2} & \color{red}{0 < x \leq 1 , t > 0}\\
\end{align*}
$$

Initial conditions:

$$
\begin{align*}
y(0, x) & = 0 & 0 < x \color{red}{<} 1 \\
\end{align*}
$$

Boundary conditions:

$$
\begin{align*}
y(t, 0) & = Y_s & t \geq 0 \\
\frac{dy}{dx}(t, 1) & = 0 & t \geq 0 \\
\end{align*}
$$

### Using collocation

PDE

$$
\begin{align*}
\alpha \frac{\partial y}{\partial t} & = \beta \frac{\partial^2 y}{\partial x^2} & \color{red}{0 < x < 1 , t > 0}\\
\end{align*}
$$

Initial conditions:

$$
\begin{align*}
y(0, x) & = 0 & 0 < x \color{red}{<} 1 \\
\end{align*}
$$

Boundary conditions:

$$
\begin{align*}
y(t, 0) & = Y_s & t \geq 0 \\
\frac{dy}{dx}(t, 1) & = 0 & t \geq 0 \\
\end{align*}
$$

In [ ]:
def pde_model_r(discretizer="finite_difference"):

    # parameters
    tf = 1.00
    Ys = 10.0
    alpha = 5.0
    beta = 5.0

    m = pyo.ConcreteModel()

    m.t = dae.ContinuousSet(bounds=(0, tf))
    m.x = dae.ContinuousSet(bounds=(0, 1))

    m.y = pyo.Var(m.t, m.x)
    m.s = pyo.Var(m.t, m.x)

    m.dydt = dae.DerivativeVar(m.y, wrt=m.t)
    m.dydx = dae.DerivativeVar(m.y, wrt=m.x)
    m.d2ydx2 = dae.DerivativeVar(m.y, wrt=(m.x, m.x))

    @m.Constraint(m.t, m.x)
    def pde(m, t, x):
        if discretizer == "finite_difference":
            pde_deactivate_set = (x == 0) or (t == 0)
        else:
            pde_deactivate_set = (x == 0) or (x == 1) or (t == 0)

        if pde_deactivate_set:
            return pyo.Constraint.Skip
        else:
            return beta * m.dydt[t, x]  == alpha *  m.d2ydx2[t, x]

    @m.Constraint(m.t)
    def bc1(m, t):
        return m.y[t, 0] == 0

    @m.Constraint(m.t)
    def bc2(m, t):
        return m.dydx[t, 1] == 0

    @m.Constraint(m.x)
    def ic1(m, x):
        if x == 0 or x == 1:
            return pyo.Constraint.Skip
        return m.y[0, x] == 0.0

    # transform and solve
    if discretizer == "finite_difference":
        pyo.TransformationFactory('dae.finite_difference').apply_to(m, wrt=m.x, nfe=20)
        pyo.TransformationFactory('dae.finite_difference').apply_to(m, wrt=m.t, nfe=30)
    else:
        pyo.TransformationFactory('dae.collocation').apply_to(m, wrt=m.x, nfe=20, ncp=3)
        pyo.TransformationFactory('dae.collocation').apply_to(m, wrt=m.t, nfe=30, ncp=3)

    return m

In [ ]:
model = pde_model_r()
pyo.SolverFactory('ipopt').solve(model).write()
dt = DiagnosticsToolbox(model)
dt.report_structural_issues()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 2520
  Number of variables: 2520
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.16434454917907715
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Mo

In [ ]:
model = pde_model_r("collocation")
pyo.SolverFactory('ipopt').solve(model).write()
dt = DiagnosticsToolbox(model)
dt.report_structural_issues()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 21961
  Number of variables: 21961
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.7349197864532471
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
M